In [1]:
%display latex

In [2]:
lambda_1, lambda_2, lambda_3 = var("lambda_1 lambda_2 lambda_3", domain="real")


def psi(r, n=2):
    assert 1 <= n <= 3
    ret = 1 + lambda_1 * r ^ 2
    if n >= 2:
        ret += lambda_2 * r ^ 4
    if n == 3:
        ret += lambda_3 * r ^ 6
    return ret


psi(var("r"))

lambda_2*r^4 + lambda_1*r^2 + 1

In [3]:
res = vector([1200, 800])
max_point = vector([950, 630])

max_r = norm(max_point - res / 2)  # non-distorted
max_r_hat = norm(res / 2)  # distorted
max_r, float(max_r), max_r_hat, float(max_r_hat)

(10*sqrt(1754), 418.80783182743846, 200*sqrt(13), 721.1102550927978)

In [4]:
ineqs = [0 < psi(max_r_hat), psi(max_r_hat) < max_r]
ineqs

[0 < 520000*lambda_1 + 270400000000*lambda_2 + 1,
 520000*lambda_1 + 270400000000*lambda_2 + 1 < 10*sqrt(1754)]

In [5]:
sol = solve(ineqs, (lambda_1, lambda_2))
assert len(sol) == 1
sol[0]

[-520000*lambda_2 - 1/520000 < lambda_1,
 lambda_1 < -520000*lambda_2 + 1/52000*sqrt(1754) - 1/520000]

In [6]:
def approx_rationals(expr):
    if expr.operator() is None:  # Reached a leaf node (number or variable)
        return numerical_approx(expr) if expr in QQ else expr
    else:
        return expr.operator()(
            *[approx_rationals(operand) for operand in expr.operands()]
        )


for s in sol[0]:
    display(approx_rationals(s))

-520000.000000000*lambda_2 - 1.92307692307692e-6 < lambda_1

lambda_1 < -520000.000000000*lambda_2 + 0.000803476599668151

In [7]:
for s in solve(sol[0], lambda_2)[0]:
    display(float(1e6) * approx_rationals(s))

-1.92307692307692*lambda_1 - 3.69822485207101e-6 < 1000000.0*lambda_2

1000000.0*lambda_2 < -1.92307692307692*lambda_1 + 0.00154514730705414

In [8]:
for l1 in [-5, -1, 0, 1, 5]:
    eq = lambda_1 == l1
    display(eq)
    for s in solve(sol[0], lambda_2)[0]:
        display(approx_rationals(s).subs(eq))
    print(
        f"lambda_2 range: {(sol[0][1].subs(eq).rhs() - sol[0][0].subs(eq).lhs()).n()}"
    )
    print()

lambda_1 == -5

(9.61538091715976e-6) < lambda_2

lambda_2 < (9.61692976269167e-6)

lambda_2 range: 0.000805399676591228



lambda_1 == -1

...

lambda_2 range: 0.000805399676591228



lambda_1 == 5

(-9.61538831360947e-6) < lambda_2

lambda_2 < (-9.61383946807756e-6)

lambda_2 range: 0.000805399676591228



In [9]:
for s in solve(sol[0], lambda_2)[0]:
    print(float(1e6) * approx_rationals(s))

-1.92307692307692*lambda_1 - 3.69822485207101e-6 < 1000000.0*lambda_2
1000000.0*lambda_2 < -1.92307692307692*lambda_1 + 0.00154514730705414
